In [3]:
import torch
import argparse

import os
import sys

import numpy as np
import pickle 

# import trainer
# -----------------------------
# PC
# sys.path.append('/home/david/Desktop/projects/thesis/downstream_classification')
# os.chdir('/home/david/Desktop/projects/thesis/downstream_classification')

# MAC
sys.path.append('/Users/davidharar/Documents/School/thesis/git_repositories/')
os.chdir('/Users/davidharar/Documents/School/thesis/git_repositories/')

from combined_downstream_upstream.utils.LoadModels import *
from combined_downstream_upstream.executors.train_combined_model import trainer


# upstream model params and config
# ---------------
upstream_params = {
    'input_dimension': 12,
    'output_dimension': 12,              
    'hidden_dimmension':  128,           # d_model (int) – the number of expected features in the input (required)???,
    'attention_heads': 8,               # number of attention heads, if None then d_model//64,
    'encoder_number_of_layers': 8,
    'dropout': 0.4,
    'clip': 1,
    'positional_encodings': False,
    'device':'mps'
}

# run script
# -----------------------------
best_rocauc_and_pr_auc = {
    'channels_to_turn_off':[],
    'seed':[],
    'validation-roc-auc':[],
    'validation-pr-auc':[],
    'test-roc-auc':[],
    'test-pr-auc':[],
}

seed = 123
np.random.seed(123)
seeds = np.random.randint(0,1000, 15)

config = {
            # general
            'targets': ['AF'],
            'leads': ['LI', 'LII', 'LIII', 'aVL', 'aVR','aVF', 'V1','V2','V3','V4','V5','V6'],
            # 'leads': ['LI', 'LII', 'LIII', 'aVF', 'aVL', 'aVR','V1','V2','V3','V4','V5','V6'],

            # training
            'batch_size': 128,
            'n_epochs': 10, 
            'weight_decay': 0.3,
            'lr': 0.0005,
            'eval_metric':'aucpr',
            'patience':3,
            'clip':1,
            'loss_function_weight':None,
            'predefined_device':'mps',

            # Experiment settings
            'impute_only_missing':True,
            'continue_training_upstream_model':False,

            # test
            'check_on_test':True,
            'plot':False, # convert to true if want to plot
            }



seed_ = 42

# Empty leads
# -------------------
config['seed']=seed_
config['metadata_file_path']    = './downstream_classification/data/combined_data/china.csv'
config['data_folder_path']      = './downstream_classification/data/china-processed_divided_into_450/'

config['internal_data']         = False
config['channels_to_turn_off']  = 0
config['model_saving_path']     = f"./combined_downstream_upstream/models/china-{config['channels_to_turn_off']}_channels_off-no_additional_noise-impute_only_missing"
config['plot_saving_path']      = f"/home/david/Desktop/projects/thesis/combined_downstream_upstream/plots/china-{config['channels_to_turn_off']}_channels_off-no_additional_noise-impute_only_missing"

upstream_model   = load_upstream_model(upstream_params, folder_path = './upstream_seq2seq/models/', model_name = 'transformer_cnn_4heads', device = upstream_params['device'])
upstream_model

Upstream Model File Path: ./upstream_seq2seq/models/transformer_cnn_4heads_1701990699.9536803.pt
Num attention heads: 8


TSTransformerEncoderCNN(
  (project_input): Linear(in_features=12, out_features=128, bias=True)
  (encoder): Linear(in_features=12, out_features=128, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.4, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.4, inplace=False)
        (dropout2): Dropout(p=0.4, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=128, out_features=12, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (cnn):